In [1]:
import sys
import os
import h5py
from collections import Counter
from progressbar import *
import re
import numpy as np
import h5py
from scipy import signal
import matplotlib
from repeating_classifier_training_utils import *
from functools import reduce
from tqdm import tqdm

# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if par_dir not in sys.path:
    sys.path.append(par_dir)
%load_ext autoreload
%matplotlib inline
%autoreload 2
index_path = os.path.join(os.getcwd(), 'Index_Storage')

In [2]:
veto_path = '/fast_scratch/WatChMaL/data/IWCDmPMT_4pm_full_tank_ODveto.h5'
odv_file = h5py.File(veto_path,'r')
odv_info = {}
for key in odv_file.keys():
    odv_info[key] = np.array(odv_file[key])

In [3]:
veto_dic = {}
for i, veto in tqdm(enumerate(odv_info['veto'])):
    veto_dic[odv_info['root_files'][i], odv_info['event_ids'][i]] = veto

9000000it [00:13, 665103.31it/s]


In [7]:
event_data_file = h5py.File('/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_full_tank_pointnet.h5','r')

dset_root_files = event_data_file['root_files']
dset_event_ids = event_data_file['event_ids']

In [6]:
idx_file = np.load('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs.npz', allow_pickle=True)
test_idxs = idx_file['test_idxs']
train_idxs = idx_file['train_idxs']
val_idxs = idx_file['val_idxs']

In [6]:
test_veto = np.array([veto_dic[(dset_root_files[idx], dset_event_ids[idx])] for idx in tqdm(test_idxs)])

100%|██████████| 3351020/3351020 [21:26<00:00, 2604.16it/s]


In [7]:
np.savez(os.path.join(os.getcwd(), 'test_veto.npz'), test_veto=test_veto)

In [8]:
train_veto = np.array([veto_dic[(dset_root_files[idx], dset_event_ids[idx])] for idx in tqdm(train_idxs)])

 25%|██▍       | 1038537/4188774 [06:42<25:11, 2084.71it/s]

KeyboardInterrupt: 

In [ ]:
val_veto = np.array([veto_dic[(dset_root_files[idx], dset_event_ids[idx])] for idx in tqdm(val_idxs)])

In [ ]:
assert val_veto.shape[0] == val_idxs.shape[0]
assert train_veto.shape[0] == train_idxs.shape[0]
assert test_veto.shape[0] == test_idxs.shape[0]

In [14]:
post_veto_test_idxs=test_idxs[np.where(test_veto==0)[0]]
post_veto_train_idxs=train_idxs[np.where(train_veto==0)[0]]
post_veto_val_idxs=val_idxs[np.where(val_veto==0)[0]]

NameError: name 'train_veto' is not defined

In [ ]:
# np.savez('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_post_veto_pe_idxs.npz', train_idxs=post_veto_train_idxs, test_idxs=post_veto_test_idxs, val_idxs=post_veto_val_idxs)

In [ ]:
event_data_file.close()

In [4]:
post_veto_idx_file = np.load('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_post_veto_pe_idxs.npz',allow_pickle=True)
post_veto_test_idxs=post_veto_idx_file['test_idxs']
post_veto_train_idxs=post_veto_idx_file['train_idxs']
post_veto_val_idxs=post_veto_idx_file['val_idxs']

In [26]:
np.where(odv_info['veto']==1)[0].shape[0]

3130629

In [28]:
-(post_veto_train_idxs.shape[0] +post_veto_test_idxs.shape[0] + post_veto_val_idxs.shape[0]) + (train_idxs.shape[0] +test_idxs.shape[0] + val_idxs.shape[0]) 

2783840

## Prepare new FiTQun cut indices for post-veto test set

In [9]:
failed_idxs = np.load(os.path.join(index_path, 'fq_failed_idxs.npz'),allow_pickle=True)['failed_indices_pointing_to_h5_test_set'].astype(int)
flagged_idxs = np.load(os.path.join(index_path, 'fq_flagged_idxs.npz'),allow_pickle=True)['arr_0'].astype(int)

In [10]:
fq_cut_dict = {}
for idx in test_idxs:
    fq_cut_dict[idx] = 0
for idx in failed_idxs:
    fq_cut_dict[test_idxs[idx]] = 1
failed_removed_test_idxs = np.delete(test_idxs, failed_idxs)
for idx in flagged_idxs:
    fq_cut_dict[failed_removed_test_idxs[idx]] = 1
fq_cut_for_vetoed_set = np.array([fq_cut_dict[idx] for idx in post_veto_test_idxs])
fq_cut_idxs_for_vetoed_set = np.where(fq_cut_for_vetoed_set==1)[0]

In [11]:
fq_cut_idxs_for_vetoed_set.shape

(158664,)

In [12]:
flagged_idxs.shape

(735557,)

In [18]:
np.savez(os.path.join(index_path, 'fq_cut_idxs_for_vetoed_set.npz'),fq_cut_idxs_for_vetoed_set=fq_cut_idxs_for_vetoed_set)

In [20]:
for i, veto in enumerate(test_veto):
    fq_cut_dict[test_idxs[i]]=fq_cut_dict[test_idxs[i]] | veto
fq_plus_veto_cut = np.array([fq_cut_dict[idx] for idx in test_idxs])
fq_plus_veto_cut_idxs = np.where(fq_plus_veto_cut==1)

In [21]:
np.savez(os.path.join(index_path, 'fq_plus_veto_cut.npz'),fq_plus_veto_cut=fq_plus_veto_cut)